In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from inverse import fit_SLSQP

In [ ]:
with open("data/datasets/report.npz", 'rb') as f:
    dataset = np.load(f)
    isotherm_data = dataset["isotherm_data"]
    pore_distribution_data = dataset["pore_distribution_data"]

In [174]:
kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
pressures = np.load("data/initial kernels/Pressure_Silica.npy")
i = 711
res = fit_SLSQP(adsorption=isotherm_data[i], kernel=kernel, a_array=pore_widths, alpha=0)

In [175]:
plt.plot(pore_widths, res.x, marker='.', label="SLSQP")
plt.plot(pore_widths, pore_distribution_data[i]/pore_widths *41000, marker='.', label="FILE")
plt.legend()
plt.show()

In [176]:
isotherm_by_distribution = np.dot(kernel.T, res.x)
plt.plot(pressures, isotherm_data[i], marker='.', label="original isotherm")
plt.plot(pressures, isotherm_by_distribution, marker='.', label="isotherm by distribution")
plt.plot(pressures, np.dot(kernel.T, pore_distribution_data[i] *4000), marker='.', label="isotherm by FILE distribution")
plt.legend()
plt.show()

In [ ]:
np.sum(np.square(isotherm_by_distribution - isotherm_data[i]))

In [122]:
alpha_arr = [0, 5, 10, 20, 30, 40]
err_res = []
i = 0
for alpha in alpha_arr:
    err = 0
    for isotherm in isotherm_data[:10]:
        distr = fit_SLSQP(adsorption=isotherm, kernel=kernel, a_array=pore_widths, alpha=alpha)
        isotherm_by_distribution = np.dot(kernel.T, distr.x)
        err += np.sum(np.square(isotherm_by_distribution - isotherm))
        print(i)
        i+=1
    err_res.append(err)

C:\Users\Admin\Desktop\iso\isotherm-analyzer\inverse.py:27: RuntimeWarning: divide by zero encountered in log
  adsorption)).sum(axis=0) + alpha * np.sum(pore_dist * np.log(pore_dist)) #/ len(pore_dist) #+ beta * np.sum(
C:\Users\Admin\Desktop\iso\isotherm-analyzer\inverse.py:27: RuntimeWarning: invalid value encountered in multiply
  adsorption)).sum(axis=0) + alpha * np.sum(pore_dist * np.log(pore_dist)) #/ len(pore_dist) #+ beta * np.sum(


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [139]:
plt.plot(alpha_arr, err_res, marker=".")
plt.show()